# OpenStack_WordCount

In [ ]:
import pandas as pd
from nltk.corpus import stopwords 
import nltk
from nltk.corpus import treebank
import nltk.stem 
from nltk import stem
import numpy as np
import string
import re
import random
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [17]:
#Randomly select 1000 from bugslist and report (launch pad and stack over flow)

In [18]:
filename = "C:\\Users\\zaina\\Documents\\DataVisulaization_MachineLearning\\Open_Stack\\buglist.xlsx"
df_launchpad=pd.read_excel(filename)
filename = "C:\\Users\\zaina\\Documents\\DataVisulaization_MachineLearning\\Open_Stack\\report.xlsx"
df_stackoverflow = pd.read_excel(filename)
stop_words=stopwords.words('english')
stop_word_dict=[]
for i in stop_words:
    stop_word_dict.append(str(i.encode("utf-8")))

In [19]:
df_stackoverflow.head()

,votes,fetch_url,title,longtext,answers,link,date,id,tags,Unnamed: 9
0,0,http://stackoverflow.com/search?sort=newest&pa...,Q: Puppet Resource Ordering not working,I have a large manifest that sets up an OpenSt...,1,http://stackoverflow.com/questions/40316293/pu...,2017-01-12 19:57:35,242af36b5b43854915c3de0453e113b0,"[u'puppet', u'openstack', u'puppet-enterprise']",Cinder
1,1,http://stackoverflow.com/search?sort=newest&pa...,Q: OpenStack With Manila Installation Fails fr...,Trying to install OpenStack Kilo version with ...,1,http://stackoverflow.com/questions/31160977/op...,2017-01-12 19:57:35,ecd5d0719e0acf92f31e21c87fbdde14,"[u'installation', u'openstack', u'devstack']",Cinder
2,0,http://stackoverflow.com/search?sort=newest&pa...,A: jclouds with OpenStack => java.util.NoSuchE...,> <version>${jclouds.version}</version> </depe...,1,http://stackoverflow.com/questions/21171261/jc...,2017-01-12 19:57:35,b955a0c030dcf83897d138c842af0072,[],Cinder
3,2,http://stackoverflow.com/search?sort=newest&pa...,Q: Create an instance from volume in openstach...,I am trying to create an instance from a boota...,1,http://stackoverflow.com/questions/27048292/cr...,2017-01-12 19:57:35,5430ad88204b92f03eb5bcbd0be11fa7,"[u'python', u'openstack', u'novaclient', u'ope...",Cinder
4,1,http://stackoverflow.com/search?sort=newest&pa...,Q: Devstack: There was an error submitting the...,instance from admin user(from demo user works ...,1,http://stackoverflow.com/questions/31105705/de...,2017-01-12 19:57:36,037c08accb1cf7dc4dab038195f59116,"[u'ubuntu', u'cloud', u'openstack', u'devstack...",Cinder


In [20]:
stop_word_dict.append( '')

In [8]:
df=pd.DataFrame({'fault_description':random.sample(df_launchpad.fault_description,1000),'req':random.sample(df_launchpad.req,1000)},dtype=str)
df_st=pd.DataFrame({'longtext':random.sample(df_stackoverflow.longtext,1000),'title':random.sample(df_stackoverflow.title,1000),'tags':random.sample(df_stackoverflow.tags,1000)},dtype=str)

In [9]:
#df=pd.DataFrame({'req':random.sample(df_launchpad.req,100),'bug':random.sample(df_launchpad.bug,100),'project':random.sample(df_launchpad.project,100),'fault_description':random.sample(df_launchpad.fault_description,100),'severity':random.sample(df_launchpad.severity,100),'submitter':random.sample(df_launchpad.submitter,100),'assignee':random.sample(df_launchpad.assignee,100),'created_date':random.sample(df_launchpad.created,100)})
#df_st=pd.DataFrame({'fetch_url':random.sample(df_stackoverflow.fetch_url,100),'link':random.sample(df_stackoverflow.link,100),'date':random.sample(df_stackoverflow.date,100),'longtext':random.sample(df_stackoverflow.longtext,100),'title':random.sample(df_stackoverflow.title,100),'tags':random.sample(df_stackoverflow.tags,100),'id':random.sample(df_stackoverflow.id,100)})

In [21]:
#df=pd.DataFrame({'req':df_launchpad.req,'bug':df_launchpad.bug,'project':df_launchpad.project,'fault_description':df_launchpad.fault_description,'severity':df_launchpad.severity,'submitter':df_launchpad.submitter,'assignee':df_launchpad.assignee,'created_date':df_launchpad.created})
#df_st=pd.DataFrame({'fetch_url':df_stackoverflow.fetch_url,'link':df_stackoverflow.link,'date':df_stackoverflow.date,'longtext':df_stackoverflow.longtext,'title':df_stackoverflow.title,'tags':df_stackoverflow.tags,'id':df_stackoverflow['id']})

In [ ]:
Fault_Details_lp= df.fault_description + df.req
Fault_Details_st=pd.DataFrame(df_st.title + df_st.tags  + df_st.longtext.to_string())
df=df.assign(Fault_Details_lp=Fault_Details_lp.values)
df_st=df_st.assign(Fault_Details_st=Fault_Details_st.values)

In [22]:
#Remove non-ascii character to process data using NLTK
#NLTK coverts string into meaningful words called token, removes punctuations and special characters.
#Analyzing Fault_desc and Req field from data set
#write process data to csv file

In [ ]:
def create_dict(dataframe,field,stop_word_dict):
    corpus=[]
    for i in field:
        for k in i:
            if isinstance(k, str):
                k = unicode(k, "utf-8") 
        corpus.append(str(i.encode("utf-8")))         
    dataframe['field_corpus']=pd.Series(corpus)
    dataframe['field_corpus']=pd.Series( str(i).lower()  if len(str(i)) >0 else '' for i in dataframe.field_corpus)
    corpus=[]
    for i in dataframe.field_corpus:
        data=(str(i))
        data=re.sub(r'[^\x00-\x7f]','',data) or re.sub(r'[0-9]','',data)
        corpus.append(data) 
    dataframe.field_corpus=pd.Series(corpus)
    corpus=[]
    for i in dataframe.field_corpus:
        substr=re.split('[^A-Za-z]',i)
        for j in substr:
            j=j.strip()
            if j not in stop_word_dict and len(j)>1:
                corpus.append(j) 
        dataframe['field_corpus_words']=pd.Series(corpus)
    field_words_count={}
    print dataframe['field_corpus_words']
    for w in corpus:
        field_words_count[w]=field_words_count.get(w,0)+1 
    return field_words_count

In [ ]:
fault_words_count={}
fault_words_count=create_dict(df,df.fault_description,stop_word_dict)
req_words_count={}
req_words_count=create_dict(df,df.req,stop_word_dict)
title_words_count={}
title_words_count=create_dict(df_st,df_st.title,stop_word_dict)
tags_words_count={}
tags_words_count=create_dict(df_st,df_st.tags,stop_word_dict)
longtext_words_count={}
longtext_words_count=create_dict(df_st,df_st.longtext,stop_word_dict)

In [ ]:
'''
fault_words_count={}
fault_words_list=[]
fault_words_count,fault_words_list=create_dict(df,df.fault_description,stop_word_dict)

req_words_count={}
req_words_list=[]
req_words_count,req_words_list=create_dict(df,df.req,stop_word_dict)
    
title_words_count={}
title_words_list=[]
title_words_count,title_words_list=create_dict(df_st,df_st.title,stop_word_dict)

tags_words_count={}
tags_words_list=[]
tags_words_count,tags_words_list=create_dict(df_st,df_st.tags,stop_word_dict)

list=[]
for i in df_st.longtext:
    list.append(i)

corpus=[]
for i in list:
    for k in i:
        if isinstance(k, str):
            k = unicode(k, "utf-8") 
    corpus.append(str(i))         
df_st['longtext_corpus']=pd.Series(corpus)
df_st['longtext_corpus']=pd.Series( str(i).lower()  if len(str(i)) >0 else '' for i in df_st.longtext_corpus)

corpus=[]
for i in df_st.longtext_corpus:
    data=(str(i))
    data=re.sub(r'[^\x00-\x7f]','',data) or re.sub(r'[0-9]','',data)
    corpus.append(data) 
df_st.longtext_corpus=pd.Series(corpus)
corpus=[]
for i in df_st.longtext_corpus:
    substr=re.split('[^A-Za-z]',i)
    for j in substr:
        j=j.strip()
        if j not in stop_word_dict and len(j)>1:
            corpus.append(j) 
longtext_words_count={}
longtext_words_list=[]
for w in corpus:
    longtext_words_count[w]=longtext_words_count.get(w,0)+1
longtext_words_list=corpus'''

In [ ]:
#Creat dictionary for FaultDesc_Corpus and Req_Corpus
df_st.head()

In [ ]:
#Sorted dictionary for FaultDesc_Corpus and Req_Corpus
df.head()

In [ ]:
mylist=[]
for key, value in sorted(fault_words_count.iteritems(), key=lambda (k,v): (v,k), reverse=True):
    mylist.append([key,value])
for key, value in sorted(req_words_count.iteritems(), key=lambda (k,v): (v,k), reverse=True):
    mylist.append([key,value])
for key, value in sorted(longtext_words_count.iteritems(), key=lambda (k,v): (v,k), reverse=True):
    mylist.append([key,value])
for key, value in sorted(title_words_count.iteritems(), key=lambda (k,v): (v,k), reverse=True):
    mylist.append([key,value])
for key, value in sorted(tags_words_count.iteritems(), key=lambda (k,v): (v,k), reverse=True):
    mylist.append([key,value])
df_wordcount=pd.DataFrame(mylist, columns =['Key','Count'])

In [ ]:
len(df_wordcount)

In [ ]:
df_wordcount.to_csv('C:\\Users\\zaina\\Documents\\DataVisulaization_MachineLearning\\Open_Stack\\dictionary_sample.csv')

In [ ]:
print "dictionary created"

In [ ]:
#Using TF-IDF to convert unstructured text to useful features

<div>
    <a href="http://nlp.stanford.edu/IR-book/html/htmledition/tf-idf-weighting-1.html" target="_blank" title="nlp.stanford.edu" style="display: block; text-align: center;"><img src="http://nlp.stanford.edu/IR-book/html/htmledition/tf-idf-weighting-1.html" alt="nlp.stanford.edu" style="max-width: 100%;width: 800px;"  width="800"/></a>
</div>

In [45]:
mydict=set(df_wordcount.Key)
mydict

{'limited',
 'pacemaker',
 'prefix',
 'crete',
 'increase',
 'patches',
 'sorry',
 'spec',
 'vxlan',
 'updated',
 'every',
 'updates',
 'upstream',
 'fuelclient',
 'cause',
 'upd',
 'cmd',
 'upload',
 'rabbitmq',
 'correct',
 'convenience',
 'notifying',
 'cmp',
 'force',
 'incremented',
 'consistent',
 'direct',
 'likely',
 'consistencygroup',
 'specs',
 'even',
 'errors',
 'hide',
 'selected',
 'liberty',
 'new',
 'net',
 'metadata',
 'never',
 'met',
 'chassis',
 'codecs',
 'active',
 'path',
 'wdc',
 'changed',
 'reports',
 'ignoring',
 'orm',
 'indentation',
 'suitable',
 'changes',
 'astute',
 'txn',
 'prints',
 'qcow',
 'replace',
 'org',
 'glance',
 'txt',
 'unit',
 'would',
 'paramater',
 'commandfailure',
 'exited',
 'call',
 'therefore',
 'dns',
 'reproducibility',
 'packetary',
 'type',
 'tell',
 'vpp',
 'successful',
 'mznnldci',
 'warn',
 'defautl',
 'aligned',
 'eeae',
 'describedby',
 'must',
 'join',
 'err',
 'mb',
 'setup',
 'work',
 'doctype',
 'worth',
 'mu',
 'mv',

In [46]:
req_words_dict=dict.fromkeys(mydict,0)
title_words_dict=dict.fromkeys(mydict,0)
tags_words_dict=dict.fromkeys(mydict,0)
longtext_words_dict=dict.fromkeys(mydict,0)
fault_words_dict=dict.fromkeys(mydict,0)

In [50]:
heading=['req_words', 'title_words','tags_words','longtext_words','fault_words']
Matrix=pd.DataFrame([req_words_dict,title_words_dict,tags_words_dict,longtext_words_dict,fault_words_dict])
Temp=pd.Series(heading)
Matrix1=Matrix.copy()
Matrix1= Matrix1.assign(Temp=Temp.values)
Matrix1.index=Temp
Temp1= pd.DataFrame.transpose( Matrix1)
Temp1.to_csv('C:\\Users\\zaina\\Documents\\DataVisulaization_MachineLearning\\Open_Stack\\matrix_sample.csv')
Matrix.head()

,aa,aas,ab,aba,abb,able,ac,acceptance,access,according,...,yet,yield,ywwioiawfswgim,zabbix,zaqar,zdgfjay,zero,zone,zsfciibbtkqgdgfnczpcimnvbnnvbgvcilxuiiwgimdyyxbobw,zxj
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
def computeTF(mydict, field):
    tfDict = {}
    fieldCount= len(field)
    for key, value in mydict.iteritems():
        tfDict[key] = value/ float(fieldCount)
    return tfDict

In [61]:
tf_title_words_list = computeTF(title_words_dict, title_words_list)
tf_tags_words_list = computeTF(tags_words_dict, tags_words_list)
tf_longtext_words_list = computeTF(longtext_words_dict,longtext_words_list)
tf_fault_words_list = computeTF(fault_words_dict,fault_words_list)
tf_req_words_list = computeTF(req_words_dict,req_words_list)

NameError: name 'title_words_list' is not defined

In [58]:
def computeIDF(docList):
    import math
    idfDict= {}
    N= len(docList)
    idfDict = dict.fromkeys(docList[0].keys(),0)
    for doc in docList:
        for word, val in doc.iteritems():
            if val > 0:
                idfDict[word]+=1
    for word,val in idfDict.iteritems():
        idfDict[word] = math.log(N/ float(val))
    return idfDict

In [59]:
idfs=computeIDF([req_words_dict,title_words_dict,tags_words_dict,longtext_words_dict,fault_words_dict])

ZeroDivisionError: float division by zero

In [ ]:
idfs

In [ ]:
def computeTFIDF(tfList, idfs):
    tfidf={}
    for word,val in tfList.iteritems():
        tfidf[word]= val *  idfs[word]
    return tfidf

In [ ]:
tfidf_title_words_list = computeTFIDF(tf_title_words_list, idfs)
tfidf_tags_words_list = computeTF(tf_tags_words_list, idfs)
tfidf_longtext_words_list = computeTF(tf_longtext_words_list,idfs)
tfidf_fault_words_list = computeTF(tf_fault_words_list,idfs)
tfidf_req_words_list = computeTF(tf_req_words_list,idfs)

In [ ]:
heading=['req_words', 'title_words','tags_words','longtext_words','fault_words']
tfMatrix=pd.DataFrame([tfidf_title_words_list,tfidf_tags_words_list,tfidf_longtext_words_list,tfidf_fault_words_list,tfidf_req_words_list])
Temp=pd.Series(heading)
tfMatrix= tfMatrix.assign(Temp=Temp.values)
tfMatrix.index=Temp
tfTemp1= pd.DataFrame.transpose(tfMatrix)
tfTemp1.to_csv('C:\\Users\\zaina\\Documents\\DataVisulaization_MachineLearning\\Open_Stack\\tfmatrix.csv')
tfTemp1.head()

In [ ]:
df_st.head()

In [ ]:
df.head()

In [ ]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(df.Fault_Details_lp)

print(tfidf_matrix.shape)

In [ ]:
terms = tfidf_vectorizer.get_feature_names()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

# K-means clustering

In [ ]:
from sklearn.cluster import KMeans
num_clusters = 5
km = KMeans(n_clusters=num_clusters)
%time km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

In [ ]:
from sklearn.externals import joblib
joblib.dump(km,  'C:\\Users\\zaina\\Documents\\DataVisulaization_MachineLearning\\Open_Stack\\LaundPad.pkl')

In [ ]:
km = joblib.load('C:\\Users\\zaina\\Documents\\DataVisulaization_MachineLearning\\Open_Stack\\LaundPad.pkl')
clusters = km.labels_.tolist()

In [ ]:
import pandas as pd
Cluster=pd.Series(clusters)
df=df.assign(Cluster=Cluster.values)

In [ ]:
df.head()

In [ ]:
df.to_csv('C:\\Users\\zaina\\Documents\\DataVisulaization_MachineLearning\\Open_Stack\\Classified_LaunchPad.csv')

# Word extractions

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
CountVectorizer?

In [ ]:
vocab=mylist['Key']

In [ ]:
simple_vectorizer = CountVectorizer(vocabulary=vocab)
bow1 = simple_vectorizer.fit_transform(df.Fault_Details_lp).todense()
bow2 = simple_vectorizer.fit_transform(df_st.Fault_Details_lp).todense()

In [ ]:
print bow1,bow2

In [ ]:
words = list(simple_vectorizer.vocabulary_.keys())

In [ ]:
# Convert bow to DataFrame
bow1_df = pd.DataFrame(bow1, index=df.index, columns = words)
bow2_df = pd.DataFrame(bow2, index=df_st.index, columns = words)

In [ ]:
train_data_lp= df.join(bow1_df)
train_data_st= df.join(bow2_df)